In [1]:
from tool import *

In [22]:
with open('D:/Data/ODM_SR.db', 'rb') as f:
    sr = pickle.load(f)
with open('D:/Data/sitemap.db', 'rb') as f:
    site = pickle.load(f)
with open('D:/Data/contact_window.db', 'rb') as f:
    cont = pickle.load(f)

In [23]:
thisweek = get_weekname(datetime.date.today())

In [24]:
sr['Week Name'] = sr['Ship Date'].map(get_weekname)

In [25]:
sr = sr.reset_index().merge(site[['To Site', 'Region', 'Country', 'Subsidiary']])

In [26]:
sr.loc[sr['Division']=='PC', 'SCM'] = sr.loc[sr['Division']=='PC', 'Subsidiary'].map(cont[(cont['Div']=='PC') & (cont['Team']=='SCM')].groupby(['Subsidiary'])['Name'].max())
sr.loc[sr['Division']=='PC', 'MKT'] = sr.loc[sr['Division']=='PC', 'Subsidiary'].map(cont[(cont['Div']=='PC') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].max())
sr.loc[sr['Division']=='MNT', 'SCM'] = sr.loc[sr['Division']=='MNT', 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='SCM')].groupby(['Subsidiary'])['Name'].max())
sr.loc[sr['Division']=='MNT', 'MKT'] = sr.loc[sr['Division']=='MNT', 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].min())
us_b2b = sr.loc[sr['To Site'].str.contains('ENUS') & (sr['Mapping Model.Suffix'].apply(lambda x:x[2]) == 'B'), 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].max())
sr.loc[sr['To Site'].str.contains('ENUS') & (sr['Mapping Model.Suffix'].apply(lambda x:x[2]) == 'B'), 'MKT'] = us_b2b

In [27]:
sr = sr[sr['Week Name']==get_weekname_from(thisweek, -1)].fillna('N/A').pivot_table('Ship', index=['From Site', 'BL No', 'Region', 'Country', 'Subsidiary', 'To Site', 'Mapping Model.Suffix', 'PO No.', 'PO1 No', 'SO No.', 'Final Destination', 'Ship Date', 'SCM', 'MKT'], aggfunc=sum).reset_index()
# sr = sr[sr['Week Name'].isin([get_weekname_from(thisweek, 0)])].fillna('N/A').pivot_table('Ship', index=['From Site', 'BL No', 'Region', 'Country', 'Subsidiary', 'To Site', 'Mapping Model.Suffix', 'PO No.', 'PO1 No', 'SO No.', 'Final Destination', 'Ship Date', 'SCM', 'MKT'], aggfunc=sum).reset_index()

In [28]:
sr = sr[sr['From Site'].str.contains('TCL')]

# 매출 담당자
### 1) EU : 전유미 선임
### 2) MEA : 오설 책임
### 3) SEA : 현은지 선임
### 4) NA : 김정미 사원
### 5) SCA : 최경남 책임
### 6) Korea : 이창훈 책임, 김윤정 선임, 김준현 사원, 조희진 선임, 송예슬 사원, 김승현 기정, 양서하 사원, 김운 책임, 정경훈 조장, 이태균 선임, 김종철 책임

#### 이창훈 책임 changhoon123.lee.lee@lgepartner.com
#### 조희진 선임 hyijin.cho@lgepartner.com
#### 송예슬 사원 yeseul.song@lgepartner.com

In [29]:
sr

,From Site,BL No,Region,Country,Subsidiary,To Site,Mapping Model.Suffix,PO No.,PO1 No,SO No.,Final Destination,Ship Date,SCM,MKT,Ship
124,OS_TCL_CN_P,PLISZ4F00807,MEA,Israel,LGEYK,EMYK_HYE_DO,24MR400-B.AMFQ,JGMZ23037280,1061831,380173560,ILASH,2024-01-22,오석현,이성호,320.0
125,OS_TCL_CN_P,PLISZ4F00807,MEA,Israel,LGEYK,EMYK_HYE_DO,27MR400-B.AMFQ,JGMZ23037279,1061831,380173560,ILASH,2024-01-22,오석현,이성호,256.0
126,OS_TCL_CN_P,PLISZ4F00950,MEA,Saudi Arabia,LGESJ,EMSJ,24MR400-B.AMIQ,JGMZ23037197,1002168,380173488,SARUH,2024-01-23,오석현,이성호,800.0
127,OS_TCL_CN_P,PLISZ4F00950,MEA,Saudi Arabia,LGESJ,EMSJ,27MR400-B.AMIQ,JGMZ23037198,1002168,380173488,SARUH,2024-01-23,오석현,이성호,512.0
128,OS_TCL_CN_P,PLISZ4F00957,MEA,UAE,LGEGF,EMGF_ECHODUBAI_DO,22MR410-B.AMAQ,JGMZ24000647,1195477,380174086,AEJEA,2024-01-23,오석현,최성용,1680.0
129,OS_TCL_CN_P,PLISZ4F00957,MEA,UAE,LGEGF,EMGF_ECHODUBAI_DO,24MR400-B.AMAQ,JGMZ24000646,1195477,380174086,AEJEA,2024-01-23,오석현,최성용,1600.0
130,OS_TCL_CN_P,PLISZ4F00957,MEA,UAE,LGEGF,EMGF_ECHODUBAI_DO,27MR400-B.AMAQ,JGMZ24000652,1195498,380174088,AEJEA,2024-01-23,오석현,최성용,1280.0
131,OS_TCL_CN_P,PLISZ4F00965,SEA,Indonesia,LGEIN,EAIN_DC,22MR410-B.ATIQ,JGMZ23037196,833209,380173487,IDCAK,2024-01-24,김송이,임채구,2184.0
132,OS_TCL_CN_P,PLISZ4F01004,SEA,New Zealand,LGEAP,EHAP_NZA,24MR400-B.AAUQ,JGMZ23037202,1106774,380173491,NZAKL,2024-01-23,김송이,최순필,1248.0
133,OS_TCL_CN_P,PLISZ4F01004,SEA,New Zealand,LGEAP,EHAP_NZA,27MR400-B.AAUQ,JGMZ23037201,1106774,380173491,NZAKL,2024-01-23,김송이,최순필,480.0


In [10]:
bl_num = ['PLISZ4F01035']
c1 = sr['BL No'].isin(bl_num)

In [11]:
sr[c1].to_clipboard(index=False)
sr[c1]

,From Site,BL No,Region,Country,Subsidiary,To Site,Mapping Model.Suffix,PO No.,PO1 No,SO No.,Final Destination,Ship Date,SCM,MKT,Ship
137,OS_TCL_CN_P,PLISZ4F01035,EU,Portugal,LGEPT,EEPT,27MR400-B.AEUQ,JGMZ23037190,1031779,380173475,PTAZA,2024-01-27,송명규,조홍철,512.0
138,OS_TCL_CN_P,PLISZ4F01035,EU,Portugal,LGEPT,EEPT,27MR400-B.AEUQ,JGMZ24000938,1031824,380174223,PTAZA,2024-01-27,송명규,조홍철,512.0


In [12]:
# bl_num = sr.loc[i]['BL No']
region = sr[c1]['Region'].unique()[0]
country = sr[c1]['Country'].unique()[0]
weekname = sr[c1]['Ship Date'].map(get_weekname).unique()[0]

email_subject = f'제목 : 중국 ODM社 TCL 모니터 제품 중계무역 {region}향({country}) 매출화 요청 {bl_num}'
email_body = f'''안녕하십니까?

아래의 건으로 {weekname}주차 {region} 지역인 {country}향 중계무역 매출화 요청 드립니다.
상세한 내용은 첨부의 B/L 참조하시기 바랍니다.
'''
email_subject_kor = f'제목 : 중국 ODM社 TCL 모니터 제품 한국향 통관 요청 {bl_num}'
email_body_kor = f'''
안녕하십니까?

수신 : Pantos 이창훈 책임님
아래의 건으로 중국 모니터 업체인 TCL사로부터 {weekname}주차 한국으로 선적된 모니터 제품 Port에 도착되면 통관 진행 요청 드립니다.
상세한 내용은 첨부의 선적 서류 참조하시기 바랍니다.

양서하 사원님

CC request 부탁 드립니다.

정경훈 조장님

입고 검사 진행해주시기 바랍니다.

'''    

In [13]:
if sr[c1]['Country'].values[0] == 'Korea':
    print(email_subject_kor)
    print(email_body_kor)
elif region == 'SEA':
    print(email_subject)
    print(email_body)
    nw = pd.read_excel(get_filename(), skiprows=1)['Net Wt'].dropna().values[-1]
    print(f'Net Weight: {nw} KG')
else:
    print(email_subject)
    print(email_body)

제목 : 중국 ODM社 TCL 모니터 제품 중계무역 EU향(Portugal) 매출화 요청 ['PLISZ4F01035']
안녕하십니까?

아래의 건으로 2024-01-22(W04)주차 EU 지역인 Portugal향 중계무역 매출화 요청 드립니다.
상세한 내용은 첨부의 B/L 참조하시기 바랍니다.



## 인도법인 매출 시 수신자 추가

kushik.kaushal@lge.com (LGEIL IT DP)
jw20.kim@lge.com (LGEIL IT PM)
sanjeev.mittal@lge.com (LGEIL PH)
bryan1223.kim@lge.com (신규 본사담당자)

Dear EAIL team

Please see the attached shipping documents and S/N list for your reference.